In [25]:
import os
from dotenv import load_dotenv
from intake import open_catalog
import intake

import pandas as pd
import numpy as np
import geopandas as gpd

import datetime
from pystac_client import Client as psc

#import dask.distributed
import stackstac

from pyproj import CRS
from pyproj import Transformer

In [30]:
# Variables à changer pour récupérer les données sources de la chaîne des feux dans le système de l'oeil
# Utiliser intake et dotenv 
# Intake pour ouvrir le catalogue à partir de la db postgres de l'OEIL
# dotenv pour gérer les credentials et les chemins d'accès

PATH_GEOM = r"E:\FILES\OEIL\datas\sentinel_surfaces_detectees_sept_oct_2023.gpkg"

URL = "https://earth-search.aws.element84.com/v1"



crs_rgnc = CRS.from_epsg(3163)
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(crs_rgnc, crs_4326)

# On lit le fichier des surfaces brûlées dans un geodataframe
ba_test = gpd.read_file(PATH_GEOM)

# On créer une nouvelle colonne pour avoir un datetime de la date
ba_test['date_']= pd.to_datetime(ba_test['date'], format='%Y-%m-%d').dt.date

# On passe de multipolygon à polygon.
ba_test = ba_test.explode()
ba_test.head(2)

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_11324\62033604.py:23: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  ba_test = ba_test.explode()


,,PLOTID,surface_id,date,nom,surface,her,commune,index,layer,path,date_,geometry
0,0,653,360203,2023-09-17,L2A_T58KGB,1.298902,A,LIFOU,NaN,surfaces_brulees_2023_sept_oct_HER_A_inf10,N:/Informatique/SIG/Etudes/2023/2309_QC_feux/P...,2023-09-17,"POLYGON ((546210.500 340354.400, 546220.500 34..."
1,0,828,360204,2023-09-20,L2A_T58KFC,1.110337,A,OUVEA,NaN,surfaces_brulees_2023_sept_oct_HER_A_inf10,N:/Informatique/SIG/Etudes/2023/2309_QC_feux/P...,2023-09-20,"POLYGON ((462460.300 417339.900, 462500.300 41..."


In [31]:
# On établit la liste des communes de notre geodataframe
liste_commune = list(set(ba_test["commune"]))

# On récupère les surfaces_id qui ont été photo interprétés, fournis par Oriane
surfaces_id = {
    "burned" : [358032, 358018, 358010, 359919, 359594, 359614, 358008, 359524, 359592, 359944],
    "unburned"  : [357997, 358001, 358002, 358017, 358012, 359543, 359788, 359498, 359545, 360203],
    "doubt": [358026, 358033,359595, 359964, 362134, 362171, 362192, 362851, 360718, 359666]
}

# On prend un intervalle de temps avec 120 jours avant la première date de détection de surface brûlée du geopackage des surfaces brûlées
datemin = (min(ba_test['date_']) - datetime.timedelta(days=120)).strftime('%Y-%m-%d') 
datemax = max(ba_test['date_']).strftime('%Y-%m-%d')

ba_test_filter = ba_test[ba_test["commune"]=="LIFOU"]
bbox = ba_test_filter["geometry"].to_crs(4326).total_bounds

dates = f'{datemin}/{datemax}'
print(f'Emprise spatiale globale des formes identifiées {bbox}')

print(f'Interval temporel {dates}')

Emprise spatiale globale des formes identifiées [167.07659632 -21.15455212 167.40841612 -20.73348055]
Interval temporel 2022-09-12/2023-10-30


In [34]:
client = psc.open(URL)
cc=50
search = client.search(
   
    collections=['sentinel-2-l2a'],
    bbox=bbox,
    datetime=dates,
    query={"eo:cloud_cover": {"lt": cc}}

    )

print(f"{search.matched()} scenes Sentinel-2 L2A trouvées dans l'interval temporel ayant - de {cc}% de couverture nuageuse")


173 scenes Sentinel-2 L2A trouvées dans l'interval temporel ayant - de 50% de couverture nuageuse


In [35]:
items = search.item_collection()

In [38]:
stack = stackstac.stack(items)
print("stack.crs",stack.crs)
bbox[0]

stack.crs epsg:32758


e:\FILES\OEIL\PYTEST\Lib\site-packages\stackstac\prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


167.07659631784472

In [39]:
sentinel_stack = stackstac.stack(
                          items,
                          bounds_latlon=[bbox[0], bbox[1],  bbox[2],  bbox[3]],

                          gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(
                               {'GDAL_HTTP_MAX_RETRY': 3,
                                'GDAL_HTTP_RETRY_DELAY': 5,
                               }),
                          epsg=4326
                          )

e:\FILES\OEIL\PYTEST\Lib\site-packages\stackstac\prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [46]:
sentinel_stack

ImportError: Dask diagnostics requirements are not installed.

Please either conda or pip install as follows:

  conda install dask                     # either conda install
  python -m pip install "dask[diagnostics]" --upgrade  # or python -m pip install

<xarray.DataArray 'stackstac-b5d35b33b63986f125ae1a302bda9de1' (time: 173,
                                                                band: 32,
                                                                y: 4611, x: 3438)>
dask.array<fetch_raster_window, shape=(173, 32, 4611, 3438), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2022-09-12...
    id                                       (time) <U24 'S2B_58KGC_20220912_...
  * band                                     (band) <U12 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 167.1 167.1 ... 167.4
  * y                                        (y) float64 -20.73 ... -21.15
    updated                                  (time) <U24 '2022-11-06T04:38:07...
    ...                                       ...
    raster:bands                             (band) object [{'nodata': 0, 'da...
    gsd    